In [ ]:
import sys
import os
sys.path.append("{}/build".format(os.getcwd()))

In [ ]:
%matplotlib ipympl

In [ ]:
from cornerstone import iHilbert, iHilbertMixD, hilbertIBoxKeys, hilbertMixDIBoxKeys, spanSfcRange, spanSfcRangeMixD, hilbertIBox, hilbertMixDIBox
import random
import matplotlib.pyplot as plt
import numpy as np

bx = 10
by = 6
bz = 2
max_level = 10
assert bx <= max_level and by <= max_level and bz <= max_level
nparticles = 200

In [ ]:
def create_random_particles(pow2_x_range, pow2_y_range, pow2_z_range, num_particles):
    """
    Generates a list of random particles within a 3D space.
    
    Parameters:
    - pow2_x_range: The range of the x-axis as a power of 2.
    - pow2_y_range: The range of the y-axis as a power of 2.
    - pow2_z_range: The range of the z-axis as a power of 2.
    - num_particles: The number of particles to generate.
    
    Returns:
    - A list of random particles within the specified 3D space.
    """
    random.seed(42)  # Set the seed to a constant value for reproducibility
    particles = []
    for _ in range(num_particles):
        x = random.randint(0, 2**pow2_x_range - 1)
        y = random.randint(0, 2**pow2_y_range - 1)
        z = random.randint(0, 2**pow2_z_range - 1)
        particles.append((x, y, z))
    return particles

def plot_particles_3d(particles):
    """
    Plots a list of particles in 3D space using matplotlib.
    
    Parameters:
    - particles: A list of particles to plot.
    """
    # Convert the list of points into separate x, y, z arrays
    x_vals, y_vals, z_vals = zip(*particles)
    
    # Plot the particles
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    max_range = max(max(x_vals) - min(x_vals), max(y_vals) - min(y_vals), max(z_vals) - min(z_vals))
    mid_x = (max(x_vals) + min(x_vals)) / 2
    mid_y = (max(y_vals) + min(y_vals)) / 2
    mid_z = (max(z_vals) + min(z_vals)) / 2

    ax.set_xlim(mid_x - max_range / 2, mid_x + max_range / 2)
    ax.set_ylim(mid_y - max_range / 2, mid_y + max_range / 2)
    ax.set_zlim(mid_z - max_range / 2, mid_z + max_range / 2)
    ax.scatter(x_vals, y_vals, z_vals, color='r')
    ax.set_title('Random Particles in 3D Space')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()

In [ ]:
particles = create_random_particles(pow2_x_range=bx, pow2_y_range=by, pow2_z_range=bz, num_particles=nparticles)
plot_particles_3d(particles)

In [ ]:
def plot_IBox_standalone(ibox):
    """
    Plots the IBox in 3D space using matplotlib.
    
    Parameters:
    - ibox: The IBox object to plot.
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Extract the min and max coordinates from the IBox
    xmin, xmax = ibox.xmin(), ibox.xmax()
    ymin, ymax = ibox.ymin(), ibox.ymax()
    zmin, zmax = ibox.zmin(), ibox.zmax()
    
    # Define the vertices of the box
    vertices = [
        [xmin, ymin, zmin],
        [xmin, ymin, zmax],
        [xmin, ymax, zmin],
        [xmin, ymax, zmax],
        [xmax, ymin, zmin],
        [xmax, ymin, zmax],
        [xmax, ymax, zmin],
        [xmax, ymax, zmax]
    ]
    
    # Define the 12 edges of the box
    edges = [
        [vertices[0], vertices[1]],
        [vertices[0], vertices[2]],
        [vertices[0], vertices[4]],
        [vertices[1], vertices[3]],
        [vertices[1], vertices[5]],
        [vertices[2], vertices[3]],
        [vertices[2], vertices[6]],
        [vertices[3], vertices[7]],
        [vertices[4], vertices[5]],
        [vertices[4], vertices[6]],
        [vertices[5], vertices[7]],
        [vertices[6], vertices[7]]
    ]
    
    # Plot the edges
    for edge in edges:
        ax.plot3D(*zip(*edge), color='b')
    
    ax.set_title('IBox in 3D Space')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()

# Function to plot IBox
def plot_IBox(ax, ibox, color='b', label='IBox'):
    xmin, xmax = ibox.xmin(), ibox.xmax()
    ymin, ymax = ibox.ymin(), ibox.ymax()
    zmin, zmax = ibox.zmin(), ibox.zmax()
    
    vertices = [
        [xmin, ymin, zmin],
        [xmin, ymin, zmax],
        [xmin, ymax, zmin],
        [xmin, ymax, zmax],
        [xmax, ymin, zmin],
        [xmax, ymin, zmax],
        [xmax, ymax, zmin],
        [xmax, ymax, zmax]
    ]
    
    edges = [
        [vertices[0], vertices[1]],
        [vertices[0], vertices[2]],
        [vertices[0], vertices[4]],
        [vertices[1], vertices[3]],
        [vertices[1], vertices[5]],
        [vertices[2], vertices[3]],
        [vertices[2], vertices[6]],
        [vertices[3], vertices[7]],
        [vertices[4], vertices[5]],
        [vertices[4], vertices[6]],
        [vertices[5], vertices[7]],
        [vertices[6], vertices[7]]
    ]
    
    for edge in edges:
        ax.plot3D(*zip(*edge), color=color)

In [ ]:
def sort_particles_by_key(particles, origin=(0, 0, 0)):
    """
    Sorts a list of particles based on their distance from a given origin.
    
    Parameters:
    - particles: A list of particles to sort.
    - origin: The origin point to calculate distances from (default is (0, 0, 0)).
    
    Returns:
    - A list of particles sorted by their distance from the origin.
    """
    return sorted(particles, key=lambda p: iHilbertMixD(p[0], p[1], p[2], bx, by, bz))

sorted_particles = sort_particles_by_key(particles)
for i, particle in enumerate(sorted_particles):
    print(f"Particle {i}: {particle} -> MixD: {oct(iHilbertMixD(particle[0], particle[1], particle[2], bx, by, bz))} 3D: {oct(iHilbert(particle[0], particle[1], particle[2], max_level))}")

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
level = 1
# Plot specific particles
indices_to_plot = list(range(len(sorted_particles)))
# indices_to_plot = [0, 49, 99, 149]
colors = plt.cm.jet(np.linspace(0, 1, len(indices_to_plot)))
for idx, color in zip(indices_to_plot, colors):
    particle = sorted_particles[idx]
    particle_key = iHilbert(particle[0], particle[1], particle[2], max_level)
    box = hilbertIBox(particle_key, level)
    plot_IBox(ax, box, color=color, label=f'IBox_{idx}')
    ax.text(box.xmin(), box.ymin(), box.zmin(), f'IBox_{idx}', color=color)
    ax.scatter(*particle, color=color, label=f'Particle {idx}')


ax.legend()
plt.show()

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
for idx, color in zip(indices_to_plot, colors):
    particle = sorted_particles[idx]
    particle_key = iHilbertMixD(particle[0], particle[1], particle[2], bx, by, bz)
    box = hilbertMixDIBox(particle_key, max_level - level, bx, by, bz)
    plot_IBox(ax, box, color=color, label=f'IBox_{idx}')
    ax.text(box.xmin(), box.ymin(), box.zmin(), f'IBox_{idx}', color=color)
    ax.scatter(*particle, color=color, label=f'Particle {idx}')

ax.legend()
plt.show()



In [ ]:
pa = sorted_particles[2]
pb = sorted_particles[50]

print(pa, pb)

In [ ]:
pa_iHilbert_key = iHilbert(pa[0], pa[1], pa[2], max_level)
pb_iHilbert_key = iHilbert(pb[0], pb[1], pb[2], max_level)

print("Particle A iHilbert key: {}\toct: {}\tbin: {}".format(pa_iHilbert_key, oct(pa_iHilbert_key), bin(pa_iHilbert_key)))
print("Partic;e B iHilbert key: {}\toct: {}\tbin: {}".format(pb_iHilbert_key, oct(pb_iHilbert_key), bin(pb_iHilbert_key)))

num_values, span_sfc = spanSfcRange(0, pa_iHilbert_key)

print("Number of values between A and B:", num_values)
print("Octal values of span_sfc:")
print("\n".join([oct(key) for key in span_sfc]))

In [ ]:
pa_iHilbertMixD_key = iHilbertMixD(pa[0], pa[1], pa[2], bx, by, bz)
pb_iHilbertMixD_key = iHilbertMixD(pb[0], pb[1], pb[2], bx, by, bz)

print("Particle A iHilbert key: {}\toct: {}\tbin: {}".format(pa_iHilbertMixD_key, oct(pa_iHilbertMixD_key), bin(pa_iHilbertMixD_key)))
print("Partic;e B iHilbert key: {}\toct: {}\tbin: {}".format(pb_iHilbertMixD_key, oct(pb_iHilbertMixD_key), bin(pb_iHilbertMixD_key)))

num_values_mixd, span_sfc_mixd = spanSfcRange(0, pa_iHilbertMixD_key)

print("Number of values between A and B:", num_values_mixd)
# print("Octal values of span_sfc:")
# print("\n".join([oct(key) for key in span_sfc_mixd]))

In [ ]:
print("Particle A iHilbert key: {}\toct: {}\tbin: {}".format(pa_iHilbertMixD_key, oct(pa_iHilbertMixD_key), bin(pa_iHilbertMixD_key)))
print("Partic;e B iHilbert key: {}\toct: {}\tbin: {}".format(pb_iHilbertMixD_key, oct(pb_iHilbertMixD_key), bin(pb_iHilbertMixD_key)))

num_values_mixd, span_sfc_mixd = spanSfcRangeMixD(min(pa_iHilbertMixD_key, pb_iHilbertMixD_key), max(pa_iHilbertMixD_key, pb_iHilbertMixD_key), bx, by, bz)

print("Number of values between A and B:", num_values_mixd)
print("Octal values of span_sfc:")
print("\n".join([oct(key) for key in span_sfc_mixd]))

In [ ]:
# _, span_3D_sfc_mixd_start = spanSfcRange(0, max(pa_iHilbertMixD_key, pb_iHilbertMixD_key))
# _, span_3D_sfc_mixd_end = spanSfcRange(max(pa_iHilbertMixD_key, pb_iHilbertMixD_key), int('10000000', 8))
# _, span_3D_sfc_mixd_end_end = spanSfcRange(int('10000000', 8), int('10000000000', 8))
# for key in span_3D_sfc_mixd_start:
#     print(oct(key))
# for key in span_3D_sfc_mixd_end:
#     print(oct(key))
# for key in span_3D_sfc_mixd_end_end:
#     print(oct(key))

In [ ]:
pabIBox = hilbertIBoxKeys(pa_iHilbert_key, pa_iHilbert_key+1)

plot_IBox_standalone(pabIBox)

In [ ]:
pabIBoxMixD = hilbertMixDIBoxKeys(pa_iHilbertMixD_key, pa_iHilbertMixD_key+1, bx, by, bz)

plot_IBox_standalone(pabIBoxMixD)

In [ ]:
def plot_combined(pa, pb, pabIBox, pabIBoxMixD):
    """
    Plots the points pa, pb and the IBoxes pabIBox and pabIBoxMixD in 3D space using matplotlib.
    
    Parameters:
    - pa: The first particle point.
    - pb: The second particle point.
    - pabIBox: The first IBox object to plot.
    - pabIBoxMixD: The second IBox object to plot.
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot pa and pb
    ax.scatter(*pa, color='r', label='Particle A')
    ax.scatter(*pb, color='g', label='Particle B')
    
    # Function to plot IBox
    def plot_IBox(ax, ibox, color='b', label='IBox'):
        xmin, xmax = ibox.xmin(), ibox.xmax()
        ymin, ymax = ibox.ymin(), ibox.ymax()
        zmin, zmax = ibox.zmin(), ibox.zmax()
        
        vertices = [
            [xmin, ymin, zmin],
            [xmin, ymin, zmax],
            [xmin, ymax, zmin],
            [xmin, ymax, zmax],
            [xmax, ymin, zmin],
            [xmax, ymin, zmax],
            [xmax, ymax, zmin],
            [xmax, ymax, zmax]
        ]
        
        edges = [
            [vertices[0], vertices[1]],
            [vertices[0], vertices[2]],
            [vertices[0], vertices[4]],
            [vertices[1], vertices[3]],
            [vertices[1], vertices[5]],
            [vertices[2], vertices[3]],
            [vertices[2], vertices[6]],
            [vertices[3], vertices[7]],
            [vertices[4], vertices[5]],
            [vertices[4], vertices[6]],
            [vertices[5], vertices[7]],
            [vertices[6], vertices[7]]
        ]
        
        for edge in edges:
            ax.plot3D(*zip(*edge), color=color)
    
    # Plot IBoxes
    plot_IBox(ax, pabIBox, color='b')
    ax.text(pabIBox.xmin(), pabIBox.ymin(), pabIBox.zmax(), 'IBox', color='b')
    
    plot_IBox(ax, pabIBoxMixD, color='m')
    ax.text(pabIBoxMixD.xmax(), pabIBoxMixD.ymax(), pabIBoxMixD.zmin(), 'IBoxMixD', color='m')
    
    ax.set_title('Particles and IBoxes in 3D Space')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    plt.show()

plot_combined(pa, pb, pabIBox, pabIBoxMixD)

In [ ]:
def plot_combined(pa, pb, particles, pabIBox, pabIBoxMixD):
    """
    Plots the points pa, pb and the IBoxes pabIBox and pabIBoxMixD in 3D space using matplotlib.
    
    Parameters:
    - pa: The first particle point.
    - pb: The second particle point.
    - pabIBox: The first IBox object to plot.
    - pabIBoxMixD: The second IBox object to plot.
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot pa and pb
    ax.scatter(*pa, color='r', label='Particle A')
    ax.scatter(*pb, color='g', label='Particle B')
    
    def plot_particles_3d(ax, particles):
        """
        Plots a list of particles in 3D space using matplotlib.
        
        Parameters:
        - particles: A list of particles to plot.
        """
        # Convert the list of points into separate x, y, z arrays
        x_vals, y_vals, z_vals = zip(*particles)
        
        # Plot the particles
        max_range = max(max(x_vals) - min(x_vals), max(y_vals) - min(y_vals), max(z_vals) - min(z_vals))
        mid_x = (max(x_vals) + min(x_vals)) / 2
        mid_y = (max(y_vals) + min(y_vals)) / 2
        mid_z = (max(z_vals) + min(z_vals)) / 2

        ax.set_xlim(mid_x - max_range / 2, mid_x + max_range / 2)
        ax.set_ylim(mid_y - max_range / 2, mid_y + max_range / 2)
        ax.set_zlim(mid_z - max_range / 2, mid_z + max_range / 2)
        ax.scatter(x_vals, y_vals, z_vals, color='gray')
        ax.set_title('Random Particles in 3D Space')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')

    # Function to plot IBox
    def plot_IBox(ax, ibox, color='b', label='IBox'):
        xmin, xmax = ibox.xmin(), ibox.xmax()
        ymin, ymax = ibox.ymin(), ibox.ymax()
        zmin, zmax = ibox.zmin(), ibox.zmax()
        
        vertices = [
            [xmin, ymin, zmin],
            [xmin, ymin, zmax],
            [xmin, ymax, zmin],
            [xmin, ymax, zmax],
            [xmax, ymin, zmin],
            [xmax, ymin, zmax],
            [xmax, ymax, zmin],
            [xmax, ymax, zmax]
        ]
        
        edges = [
            [vertices[0], vertices[1]],
            [vertices[0], vertices[2]],
            [vertices[0], vertices[4]],
            [vertices[1], vertices[3]],
            [vertices[1], vertices[5]],
            [vertices[2], vertices[3]],
            [vertices[2], vertices[6]],
            [vertices[3], vertices[7]],
            [vertices[4], vertices[5]],
            [vertices[4], vertices[6]],
            [vertices[5], vertices[7]],
            [vertices[6], vertices[7]]
        ]
        
        for edge in edges:
            ax.plot3D(*zip(*edge), color=color)
    
    # Plot particles
    plot_particles_3d(ax, particles)

    # Plot IBoxes
    plot_IBox(ax, pabIBox, color='b')
    ax.text(pabIBox.xmin(), pabIBox.ymin(), pabIBox.zmax(), 'IBox', color='b')
    
    plot_IBox(ax, pabIBoxMixD, color='m')
    ax.text(pabIBoxMixD.xmax(), pabIBoxMixD.ymax(), pabIBoxMixD.zmin(), 'IBoxMixD', color='m')
    
    max_range = max(ax.get_xlim()[1] - ax.get_xlim()[0], ax.get_ylim()[1] - ax.get_ylim()[0], ax.get_zlim()[1] - ax.get_zlim()[0])
    mid_x = (ax.get_xlim()[1] + ax.get_xlim()[0]) / 2
    mid_y = (ax.get_ylim()[1] + ax.get_ylim()[0]) / 2
    mid_z = (ax.get_zlim()[1] + ax.get_zlim()[0]) / 2

    ax.set_xlim(mid_x - max_range / 2, mid_x + max_range / 2)
    ax.set_ylim(mid_y - max_range / 2, mid_y + max_range / 2)
    ax.set_zlim(mid_z - max_range / 2, mid_z + max_range / 2)

    ax.set_title('Particles and IBoxes in 3D Space')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    plt.show()

In [ ]:
plot_combined(pa, pb, particles, pabIBox, pabIBoxMixD)

In [ ]:
def plot_IBox(ax, ibox, color='b'):
        xmin, xmax = ibox.xmin(), ibox.xmax()
        ymin, ymax = ibox.ymin(), ibox.ymax()
        zmin, zmax = ibox.zmin(), ibox.zmax()
        
        vertices = [
            [xmin, ymin, zmin],
            [xmin, ymin, zmax],
            [xmin, ymax, zmin],
            [xmin, ymax, zmax],
            [xmax, ymin, zmin],
            [xmax, ymin, zmax],
            [xmax, ymax, zmin],
            [xmax, ymax, zmax]
        ]
        
        edges = [
            [vertices[0], vertices[1]],
            [vertices[0], vertices[2]],
            [vertices[0], vertices[4]],
            [vertices[1], vertices[3]],
            [vertices[1], vertices[5]],
            [vertices[2], vertices[3]],
            [vertices[2], vertices[6]],
            [vertices[3], vertices[7]],
            [vertices[4], vertices[5]],
            [vertices[4], vertices[6]],
            [vertices[5], vertices[7]],
            [vertices[6], vertices[7]]
        ]
        
        for edge in edges:
            ax.plot3D(*zip(*edge), color=color)

In [ ]:
span_sfc_mixd_plus_pkey = span_sfc_mixd
span_sfc_mixd_plus_pkey.append(max(pa_iHilbertMixD_key, pb_iHilbertMixD_key))
print("Octal values of span_sfc_mixd_plus_pkey:")
print("\n".join([oct(key) for key in span_sfc_mixd_plus_pkey]))
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
for i in range(len(span_sfc_mixd_plus_pkey) - 1):
    box = hilbertMixDIBoxKeys(span_sfc_mixd_plus_pkey[i], span_sfc_mixd_plus_pkey[i + 1], bx, by, bz)
    print(f"Box {oct(span_sfc_mixd_plus_pkey[i])[2:]}: xmin={box.xmin()}, xmax={box.xmax()}, ymin={box.ymin()}, ymax={box.ymax()}, zmin={box.zmin()}, zmax={box.zmax()}")
    plot_IBox(ax, box, color='b')
    # ax.text(box.xmin()+(box.xmax()-box.xmin())/2, box.ymin()+(box.ymax()-box.ymin())/2, box.zmin()+(box.zmax()-box.zmin())/2, "IBox_{}".format(oct(span_sfc_mixd_plus_pkey[i])[2:]), color='b')

plot_IBox(ax, pabIBoxMixD, color='gray')
ax.text(pabIBoxMixD.xmin()+(pabIBoxMixD.xmax()-pabIBoxMixD.xmin())/2, pabIBoxMixD.ymin()+(pabIBoxMixD.ymax()-pabIBoxMixD.ymin())/2, pabIBoxMixD.zmin()+(pabIBoxMixD.zmax()-pabIBoxMixD.zmin())/2, "IBox_{}".format(oct(span_sfc_mixd_plus_pkey[-1])[2:]), color='gray')

# Set the same ticks for all axes
max_range = max(ax.get_xlim()[1] - ax.get_xlim()[0], ax.get_ylim()[1] - ax.get_ylim()[0], ax.get_zlim()[1] - ax.get_zlim()[0])
mid_x = (ax.get_xlim()[1] + ax.get_xlim()[0]) / 2
mid_y = (ax.get_ylim()[1] + ax.get_ylim()[0]) / 2
mid_z = (ax.get_zlim()[1] + ax.get_zlim()[0]) / 2

ax.set_xlim(mid_x - max_range / 2, mid_x + max_range / 2)
ax.set_ylim(mid_y - max_range / 2, mid_y + max_range / 2)
ax.set_zlim(mid_z - max_range / 2, mid_z + max_range / 2)

# Plot particles a and b
ax.scatter(*pa, color='r', label='Particle A')
ax.scatter(*pb, color='g', label='Particle B')
ax.legend()

In [ ]:
span_sfc_plus_pkey = span_sfc
span_sfc_plus_pkey.append(pa_iHilbert_key)
print("Octal values of span_sfc_plus_pkey:")
print("\n".join([oct(key) for key in span_sfc_plus_pkey]))
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
for i in range(len(span_sfc_plus_pkey) - 1):
    box = hilbertIBoxKeys(span_sfc_plus_pkey[i], span_sfc_plus_pkey[i + 1])
    plot_IBox(ax, box, color='b')
    # ax.text(box.xmin(), box.ymin(), box.zmin(), "IBox_{}".format(oct(span_sfc_plus_pkey[i])[2:]), color='b')

plot_IBox(ax, pabIBox, color='gray')
ax.text(pabIBox.xmin(), pabIBox.ymin(), pabIBox.zmin(), "IBox_{}".format(oct(span_sfc_plus_pkey[-1])[2:]), color='gray')

# Set the same ticks for all axes
max_range = max(ax.get_xlim()[1] - ax.get_xlim()[0], ax.get_ylim()[1] - ax.get_ylim()[0], ax.get_zlim()[1] - ax.get_zlim()[0])
mid_x = (ax.get_xlim()[1] + ax.get_xlim()[0]) / 2
mid_y = (ax.get_ylim()[1] + ax.get_ylim()[0]) / 2
mid_z = (ax.get_zlim()[1] + ax.get_zlim()[0]) / 2

ax.set_xlim(mid_x - max_range / 2, mid_x + max_range / 2)
ax.set_ylim(mid_y - max_range / 2, mid_y + max_range / 2)
ax.set_zlim(mid_z - max_range / 2, mid_z + max_range / 2)

# Plot particles a and b
ax.scatter(*pa, color='r', label='Particle A')
ax.scatter(*pb, color='g', label='Particle B')
ax.legend()

In [ ]:
def plot_span_sfc_mixd_boxes(bx, by, bz, keystart, keyend, particle_a=None, particle_b=None):
    """
    Plots the span of SFC mixed boxes and the total box between two keys.
    
    Parameters:
    - bx: The range of the x-axis as a power of 2.
    - by: The range of the y-axis as a power of 2.
    - bz: The range of the z-axis as a power of 2.
    - keystart: The starting key.
    - keyend: The ending key.
    - particle_a: Coordinates of the first particle (optional).
    - particle_b: Coordinates of the second particle (optional).
    """
    # Get the span of SFC mixed boxes
    num_values_mixd, span_sfc_mixd = spanSfcRangeMixD(keystart, keyend, bx, by, bz)
    span_sfc_mixd.append(keyend)

    # Plot the boxes
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    for i in range(len(span_sfc_mixd) - 1):
        box = hilbertMixDIBoxKeys(span_sfc_mixd[i], span_sfc_mixd[i + 1], bx, by, bz)
        color = colors[i % len(colors)]
        plot_IBox(ax, box, color=color)
        ax.text(box.xmin()+(box.xmax()-box.xmin())/2, box.ymin()+(box.ymax()-box.ymin())/2, box.zmin()+(box.zmax()-box.zmin())/2, "{}".format(oct(span_sfc_mixd[i])[2:]), color=color)
        # Fill the box with the same color
        ax.bar3d(box.xmin(), box.ymin(), box.zmin(), box.xmax()-box.xmin(), box.ymax()-box.ymin(), box.zmax()-box.zmin(), color=color, alpha=0.1)
    
    # Plot the total box
    total_box = hilbertMixDIBoxKeys(min(keystart, keyend), max(keystart, keyend), bx, by, bz)
    plot_IBox(ax, total_box, color='gray')
    ax.text(total_box.xmin()+(total_box.xmax()-total_box.xmin())/2, total_box.ymin()+(total_box.ymax()-total_box.ymin())/2, total_box.zmin()+(total_box.zmax()-total_box.zmin())/2, "TotalBox", color='gray')
    
    # Plot particles if given
    if particle_a:
        ax.scatter(*particle_a, color='r', label='Particle A')
    if particle_b:
        ax.scatter(*particle_b, color='g', label='Particle B')
    
    # Set the same ticks for all axes
    max_range = max(ax.get_xlim()[1] - ax.get_xlim()[0], ax.get_ylim()[1] - ax.get_ylim()[0], ax.get_zlim()[1] - ax.get_zlim()[0])
    mid_x = (ax.get_xlim()[1] + ax.get_xlim()[0]) / 2
    mid_y = (ax.get_ylim()[1] + ax.get_ylim()[0]) / 2
    mid_z = (ax.get_zlim()[1] + ax.get_zlim()[0]) / 2

    ax.set_xlim(mid_x - max_range / 2, mid_x + max_range / 2)
    ax.set_ylim(mid_y - max_range / 2, mid_y + max_range / 2)
    ax.set_zlim(mid_z - max_range / 2, mid_z + max_range / 2)
    
    ax.set_title('Span of SFC Mixed Boxes and Total Box')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    plt.show()

# Example usage
particle_a = sorted_particles[7] # wrong size of large box
particle_b = sorted_particles[23] # wrong size of large box
pa_iHilbertMixD_key = iHilbertMixD(particle_a[0], particle_a[1], particle_a[2], bx, by, bz)
print("iHilbertMixD key of particle A:", oct(pa_iHilbertMixD_key))
pb_iHilbertMixD_key = iHilbertMixD(particle_b[0], particle_b[1], particle_b[2], bx, by, bz)
print("iHilbertMixD key of particle B:", oct(pb_iHilbertMixD_key))
# pa_iHilbertMixD_key = int('1303', 8)
# pb_iHilbertMixD_key = int('1003025', 8)

plot_span_sfc_mixd_boxes(bx, by, bz, 0, pa_iHilbertMixD_key, particle_a)
print("iHilbert key of particle A:", oct(iHilbert(particle_a[0], particle_a[1], particle_a[2], max_level)))
print("iHilbert key of particle B:", oct(iHilbert(particle_b[0], particle_b[1], particle_b[2], max_level)))
print("particle_a:", particle_a)
print("particle_b:", particle_b)


In [ ]:
# for i, particle in enumerate(particles):
#     key1 = iHilbertMixD(particle[0], particle[1], particle[2], bx, by, bz)
#     for j, other_particle in enumerate(particles):
#         if i < j:
#             key2 = iHilbertMixD(other_particle[0], other_particle[1], other_particle[2], bx, by, bz)
#             box = hilbertMixDIBoxKeys(min(key1, key2), max(key1, key2), bx, by, bz)
#             num_values_mixd, span_sfc_mixd = spanSfcRangeMixD(min(key1, key2), max(key1, key2), bx, by, bz)
#             span_sfc_mixd.append(max(key1, key2))
#             for k in range(len(span_sfc_mixd) - 1):
#                 span_box = hilbertMixDIBoxKeys(span_sfc_mixd[k], span_sfc_mixd[k + 1], bx, by, bz)
#                 if (span_box.xmin() <= box.xmin() <= span_box.xmax() and
#                     span_box.ymin() <= box.ymin() <= span_box.ymax() and
#                     span_box.zmin() <= box.zmin() <= span_box.zmax() and
#                     span_box.xmin() <= box.xmax() <= span_box.xmax() and
#                     span_box.ymin() <= box.ymax() <= span_box.ymax() and
#                     span_box.zmin() <= box.zmax() <= span_box.zmax()):
#                     print(f"Particle {i} and Particle {j}: Box is included in span box {k}")
#                     # fig = plt.figure(figsize=(10, 8))
#                     # ax = fig.add_subplot(111, projection='3d')

#                     # # Plot the span boxes
#                     # for k in range(len(span_sfc_mixd) - 1):
#                     #     span_box = hilbertMixDIBoxKeys(span_sfc_mixd[k], span_sfc_mixd[k + 1], bx, by, bz)
#                     #     plot_IBox(ax, span_box, color='b')
#                     #     ax.text(span_box.xmin()+(span_box.xmax()-span_box.xmin())/2, span_box.ymin()+(span_box.ymax()-span_box.ymin())/2, span_box.zmin()+(span_box.zmax()-span_box.zmin())/2, "IBox_{}".format(oct(span_sfc_mixd[k])[2:]), color='b')

#                     # # Plot the main box
#                     # plot_IBox(ax, box, color='gray')
#                     # ax.text(box.xmin()+(box.xmax()-box.xmin())/2, box.ymin()+(box.ymax()-box.ymin())/2, box.zmin()+(box.zmax()-box.zmin())/2, "MainBox", color='gray')

#                     # # Set the same ticks for all axes
#                     # max_range = max(ax.get_xlim()[1] - ax.get_xlim()[0], ax.get_ylim()[1] - ax.get_ylim()[0], ax.get_zlim()[1] - ax.get_zlim()[0])
#                     # mid_x = (ax.get_xlim()[1] + ax.get_xlim()[0]) / 2
#                     # mid_y = (ax.get_ylim()[1] + ax.get_ylim()[0]) / 2
#                     # mid_z = (ax.get_zlim()[1] + ax.get_zlim()[0]) / 2

#                     # ax.set_xlim(mid_x - max_range / 2, mid_x + max_range / 2)
#                     # ax.set_ylim(mid_y - max_range / 2, mid_y + max_range / 2)
#                     # ax.set_zlim(mid_z - max_range / 2, mid_z + max_range / 2)

#                     # # Plot particles
#                     # ax.scatter(*particle, color='r', label='Particle')
#                     # ax.scatter(*other_particle, color='g', label='Other Particle')
#                     # ax.legend()

#                     # plt.show()
#                     # raise Exception(f"Particle {i} and Particle {j}: Box is included in span box {k}")
#             if not (box.xmin() <= other_particle[0] <= box.xmax() and
#                     box.ymin() <= other_particle[1] <= box.ymax() and
#                     box.zmin() <= other_particle[2] <= box.zmax()):
#                 print(f"Particle {i} ({particle}) and Particle {j} ({other_particle}): Box does not include the second particle")
#                 print(f"Box lengths: x_length={box.xmax() - box.xmin()}, y_length={box.ymax() - box.ymin()}, z_length={box.zmax() - box.zmin()}")

In [ ]:
pa_iHilbertMixD_key = iHilbertMixD(particle_a[0], particle_a[1], particle_a[2], bx, by, bz)
pb_iHilbertMixD_key = iHilbertMixD(particle_b[0], particle_b[1], particle_b[2], bx, by, bz)

In [ ]:
# print(iHilbertMixD(3, 4, 3, bx, by, bz))
# print(iHilbert(3, 4, 3, max_level))
# print(iHilbert(3, 0, 3, max_level))